In [1]:
from tqdm import tqdm_notebook, tnrange
from h2o.automl import H2OAutoML
from h2o.estimators.random_forest import H2ORandomForestEstimator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
import gc
import h2o
import os
DATA_PATH = './datasets/'
warnings.filterwarnings("ignore")

h2o.init()#开启h2o集群

%matplotlib

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,30 mins 51 secs
H2O cluster timezone:,Asia/Shanghai
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.0.1
H2O cluster version age:,3 months and 29 days !!!
H2O cluster name:,H2O_from_python_sjtu123_bvz1yu
H2O cluster total nodes:,1
H2O cluster free memory:,7.890 Gb
H2O cluster total cores:,20
H2O cluster allowed cores:,20
H2O cluster status:,"locked, healthy"


Using matplotlib backend: Qt5Agg


In [2]:
df_data = pd.read_csv(DATA_PATH+'df_data.csv',dtype={'first_active_month':np.str})
df_train_test_additional_features = pd.read_csv(DATA_PATH+'df_train_test_features_additional.csv')
df_additional_features = pd.read_csv(DATA_PATH+'df_additional_features.csv')

df_data = df_data.merge(df_train_test_additional_features,on='card_id',how='left')
df_data = df_data.merge(df_additional_features,on='card_id',how='left')

path = './datasets/feature2/'
sublist = os.listdir(path)
for sub in sublist:
    df = pd.read_csv(path+sub)
    df_data = df_data.merge(df,on='card_id',how='left')
    
del df_additional_features,df_train_test_additional_features,df
gc.collect()

91

In [3]:
df_tsne_fea = pd.read_csv('./datasets/df_tsne_fea.csv')
df_data = df_data.merge(df_tsne_fea,on='card_id',how='left')
df_cate_statics = pd.read_csv('./datasets/df_cate_statics.csv')
df_data = df_data.merge(df_cate_statics,on='card_id',how='left')

del df_cate_statics,df_tsne_fea
gc.collect()

28

In [4]:
df_card_merchant_statics = pd.read_csv(DATA_PATH+'df_card_merchant_statics.csv')
df_card_merchant_vec = pd.read_csv(DATA_PATH+'df_card_merchant_vec.csv')
df_card_city_statics = pd.read_csv(DATA_PATH+'df_card_city_statics.csv')

df_data = df_data.merge(df_card_merchant_vec,on='card_id',how='left')
df_data = df_data.merge(df_card_merchant_statics,on='card_id',how='left')
df_data = df_data.merge(df_card_city_statics,on='card_id',how='left')

del df_card_merchant_statics,df_card_merchant_vec,df_card_city_statics
gc.collect()

42

In [5]:
df_data.fillna(-999,inplace=True)
df_data.replace([np.inf,-1*np.inf],-999,inplace=True)

In [6]:
df_train = df_data[df_data.is_test==0]
df_test = df_data[df_data.is_test==1]
df_train['is_outlier'] = (df_train.target<-30).astype(np.int)
del df_data
gc.collect()

0

In [7]:
label = ['target']
dropCols = ['card_id','first_active_month','is_outlier','is_test','target','purchase_date','merchant_id']
tr_features = [_f for _f in df_train.columns if _f not in dropCols and df_train[_f].dtype!='object']
print(len(tr_features))

1573


In [8]:
df_train = h2o.H2OFrame.from_python(df_train)
df_test = h2o.H2OFrame.from_python(df_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
rf_reg_params = {
    'ntrees':8000,
    'max_depth':8,
    'col_sample_rate_per_tree':0.9,
    'sample_rate':0.9,
    'score_each_iteration':True,
    'stopping_metric':"rmse",
    'stopping_rounds':100,
    'keep_cross_validation_fold_assignment':True,
    'keep_cross_validation_predictions':True,
#     'fold_assignment':"stratified",
#     'fold_column':'is_outlier',
    'nfolds':5,    
    'seed':30,
}

h2o_rf = H2ORandomForestEstimator(**rf_reg_params)
h2o_rf.train(tr_features, 'target', training_frame=df_train, validation_frame=df_train)

rf_test_pred = h2o_rf.predict(test_data=df_test[tr_features]).as_data_frame()['predict'].values
rf_train_pred = h2o_rf.cross_validation_holdout_predictions().as_data_frame()['predict'].values


drf Model Build progress: |███████████████████████████████████████████████| 100%
drf prediction progress: |████████████████████████████████████████████████| 100%


In [13]:
h2o_rf.cross_validation_metrics_summary()

Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.5678713,0.0043723,1.5636142,1.5728488,1.5760108,1.5680661,1.5588169
mean_residual_deviance,13.5349655,0.2019907,13.701492,13.362196,13.674283,13.869776,13.067082
mse,13.5349655,0.2019907,13.701492,13.362196,13.674283,13.869776,13.067082
r2,0.0870693,0.0033048,0.0948414,0.0837533,0.0810235,0.0879019,0.0878261
residual_deviance,13.5349655,0.2019907,13.701492,13.362196,13.674283,13.869776,13.067082
rmse,3.6787837,0.0275330,3.7015526,3.6554337,3.6978755,3.7242148,3.614842
rmsle,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
y_target = df_train[label].as_data_frame()[label].values.reshape(-1,)
score = np.sqrt((np.sum(np.square(rf_train_pred - y_target))/df_train.shape[0]))
print("score = %s"%score)

score = 3.6790333797463406


In [12]:
df_train['oof_rf_pred_%.5f'%score] = h2o.H2OFrame.from_python(rf_train_pred)
df_test['oof_rf_pred_%.5f'%score] = h2o.H2OFrame.from_python(rf_test_pred)
h2o.download_csv(data=df_train[['card_id','oof_rf_pred_%.5f'%score]],filename='./datasets/stacking/level1/h2orf_train_pred_%.5f.csv'%score)
h2o.download_csv(data=df_test[['card_id','oof_rf_pred_%.5f'%score]],filename='./datasets/stacking/level1/h2orf_test_pred_%.5f.csv'%score)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
